<a href="https://colab.research.google.com/github/kiranmai1226/LLM/blob/main/Churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("anandshaw2001/customer-churn-dataset")

print("Path to dataset files:", path)

100%|██████████| 262k/262k [00:00<00:00, 609kB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/anandshaw2001/customer-churn-dataset/versions/1


In [10]:
import pandas as pd
import numpy as np
import os
for filename in os.listdir(path):
  if filename.endswith(".csv"):
    file_path=os.path.join(path,filename)
df=pd.read_csv(file_path)
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [12]:
print(df.isnull().sum())

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64


In [13]:
print(df.isna().sum())

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64


In [19]:
df['Exited'].value_counts()

,count
Exited,
0,7963
1,2037


In [25]:
!pip install --upgrade scikit-learn

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,roc_auc_score

features=['CreditScore','Age','Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember','EstimatedSalary']
X=df[features]
y=df['Exited']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
model=RandomForestClassifier(n_estimators=300,random_state=42,class_weight='balanced')
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(roc_auc_score(y_test,model.predict_proba(X_test)[:,1]))


0.858
              precision    recall  f1-score   support

           0       0.87      0.97      0.92      1607
           1       0.76      0.41      0.53       393

    accuracy                           0.86      2000
   macro avg       0.81      0.69      0.72      2000
weighted avg       0.85      0.86      0.84      2000

[[1556   51]
 [ 233  160]]
0.8399044574389084


In [34]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,roc_auc_score

features=['CreditScore','Age','Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember','EstimatedSalary']
X=df[features]
y=df['Exited']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [35]:
!pip install lightgbm

In [114]:
import pandas as pd
import lightgbm as lgb

train_data=lgb.Dataset(X_train,label=y_train)
test_data=lgb.Dataset(X_test,label=y_test)

scale_pos_weight=(y_train==0).sum()/(y_train==1).sum()
params={
    'objective':'binary',
    'metric':'binary_logloss',
    'boosting_type':'goss',
    'learning_rate':0.03,
    'n_estimators':200,
    'num_leaves':31,
    'scale_pos_weight':scale_pos_weight,
    'verbose':-1
}
model=lgb.train(params,train_data)
y_pred_prob=model.predict(X_test)
y_pred=(y_pred_prob>0.525).astype(int)
print(classification_report(y_test,y_pred))
print("ROC-AUC score",roc_auc_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))


/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


              precision    recall  f1-score   support

           0       0.92      0.83      0.87      1607
           1       0.50      0.71      0.59       393

    accuracy                           0.80      2000
   macro avg       0.71      0.77      0.73      2000
weighted avg       0.84      0.80      0.82      2000

ROC-AUC score 0.7675041287243627
[[1330  277]
 [ 115  278]]


In [94]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV

params = {
    'num_leaves': [31, 64, 128],
    'learning_rate': [0.01, 0.03, 0.05, 0.1],
    'n_estimators': [100, 200, 300],
    'scale_pos_weight': [1, 3, 5, 7],
    'boosting_type': ['gbdt', 'goss', 'dart']
}

clf = LGBMClassifier(random_state=42)
search = RandomizedSearchCV(clf, param_distributions=params, scoring='f1', cv=3, n_iter=30)
search.fit(X_train, y_train)

print("Best params:", search.best_params_)


Best params: {'scale_pos_weight': 3, 'num_leaves': 31, 'n_estimators': 200, 'learning_rate': 0.01, 'boosting_type': 'goss'}
